In [25]:
'''
import pandas as pd
import json
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage
import sys
import os
'''

'\nimport pandas as pd\nimport json\nfrom langchain_openai.chat_models import ChatOpenAI\nfrom langchain.prompts import ChatPromptTemplate\nfrom langchain.schema import StrOutputParser\nfrom langchain_core.messages import SystemMessage, HumanMessage\nimport sys\nimport os\n'

In [26]:
'''
# Setting up environment
sys.path.insert(0, os.path.abspath('..'))
from config import set_environment
set_environment()

#Initialize LLM
llm = ChatOpenAI(model_name='gpt-4-0613')

def load_data(csv_path, json_path):
    df = pd.read_csv(csv_path)
    with open(json_path, 'r') as f:
        articles_data = json.load(f)
    return df, articles_data
'''

"\n# Setting up environment\nsys.path.insert(0, os.path.abspath('..'))\nfrom config import set_environment\nset_environment()\n\n#Initialize LLM\nllm = ChatOpenAI(model_name='gpt-4-0613')\n\ndef load_data(csv_path, json_path):\n    df = pd.read_csv(csv_path)\n    with open(json_path, 'r') as f:\n        articles_data = json.load(f)\n    return df, articles_data\n"

In [27]:
import os
import sys
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# Setting up environment
sys.path.insert(0, os.path.abspath('..'))
from config import set_environment
set_environment()

# Initialize LLM
llm = ChatOpenAI(model_name='gpt-4-0613')

csv_path = './intermediate_data/Product_Article_Matching.csv'
df = pd.read_csv(csv_path) 


def generate_product_summary(category):
    """
    Generate a comprehensive summary for a product category
    """
    # Prompt template
    template = ChatPromptTemplate.from_messages([
        ('system', '''You are an expert technical writer specializing in electronic components and products.
         Create a comprehensive summary that includes 2-3 lines about what the product category is, how it is used, and why it is popular.
        Keep the summary concise, informative, and technical but accessible.'''),
        ('user', '''Product Category: {category}
        Please provide a summary following the specified format.''')
    ])
    
    chain = template | llm | StrOutputParser()
    summary = chain.invoke({
        "category": category,
    })
    return summary

def process_product_categories(df):

    category_column = 'Product Category'
    unique_categories = df[category_column].dropna().unique()
    
    # Generate summaries for each category
    product_summaries = {}
    
    print(f"Processing {len(unique_categories)} product categories...")
    
    for i, category in enumerate(unique_categories, 1):
            
        # Generate summary
        summary = generate_product_summary(category)
        product_summaries[category] = {
            "category": category,
            "summary": summary.strip()
        }
     # Ensure output directory exists
    os.makedirs('./output', exist_ok=True)

    # Save to JSON
    with open('./intermediate_data/Product_Description.json', 'w', encoding='utf-8') as f:
        json.dump(product_summaries, f, indent=2, ensure_ascii=False)

    print(f"Product summaries saved to {'./intermediate_data/Product_Description.json'}")
    return product_summaries


In [28]:
if __name__ == "__main__":
    summaries = process_product_categories(df)

Processing 51 product categories...
Product summaries saved to ./intermediate_data/Product_Description.json
